In [ ]:
!pip install pandas sqlalchemy pymysql openpyxl

In [5]:
import pandas as pd
import numpy as np  # NaN 처리를 위해 추가
from sqlalchemy import create_engine, text

# 1. 엑셀 파일 로드
file_path = r'C:\Users\k2003\OneDrive\문서\업무\목록수정.xlsx'
df = pd.read_excel(file_path)

# 2. 데이터 전처리
# A. 배정일자 형식 변환 (비어있으면 NaT -> None 처리)
if '배정일자' in df.columns:
    df['배정일자'] = pd.to_datetime(df['배정일자']).dt.strftime('%Y-%m-%d')

# B. 중요: 모든 NaN 값을 None으로 변환 (이 작업이 'nan' 문자열 입력을 방지합니다)
# replace를 사용하여 결측치를 파이썬의 None으로 바꿉니다.
df_update = df[['의뢰번호', '배정일자', '배정현황', '미인정']].replace({np.nan: None})

# C. 의뢰번호가 없는 행은 업데이트 대상에서 제외
df_update = df_update.dropna(subset=['의뢰번호'])

# 3. DB 연결 설정
user = "kcqt_kyj"
password = "1977519"
host = "221.155.228.179"
port = "3306"
db_name = "kcqt_qyalit"

engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{db_name}')

# 4. 업데이트 실행
try:
    with engine.begin() as conn:
        # A. 임시 테이블 생성
        conn.execute(text("""
            CREATE TEMPORARY TABLE temp_update (                 
                u_id VARCHAR(50), 
                d_val VARCHAR(20), 
                d_mal VARCHAR(100), 
                unapproved VARCHAR(100)
            )
        """))
        
        # B. 데이터 삽입 (이미 전처리를 거쳤으므로 그대로 삽입)
        data_to_insert = [
            {
                "u_id": row['의뢰번호'],
                "d_val": row['배정일자'],
                "d_mal": row['배정현황'],
                "unapproved": row['미인정']
            } for _, row in df_update.iterrows()
        ]
        
        if data_to_insert:
            conn.execute(
                text("""
                    INSERT INTO temp_update (u_id, d_val, d_mal, unapproved) 
                    VALUES (:u_id, :d_val, :d_mal, :unapproved)
                """),
                data_to_insert
            )
        
        # C. JOIN 업데이트 실행
        sql = """
            UPDATE csi_receipts AS target
            INNER JOIN temp_update AS source ON target.의뢰번호 = source.u_id
            SET target.배정일자 = source.d_val,
                target.배정현황 = source.d_mal,
                target.미인정 = source.unapproved
        """
        
        result = conn.execute(text(sql))
        print(f"✅ 업데이트 성공: 총 {result.rowcount}개의 행이 수정되었습니다. (빈 값은 NULL 처리됨)")

except Exception as e:
    print(f"❌ 오류 발생: {e}")

✅ 업데이트 성공: 총 1337개의 행이 수정되었습니다. (빈 값은 NULL 처리됨)
